In [35]:
# multi_approaches: list = ['cnn', 'lstm', 'stacked_lstm', 'bilstm',  'stacked_bilstm']
# input_window: list = [90]
# output_window: list = [30, 60, 90]

multi_approaches: list = ['stacked_bilstm']
input_window: list = [90]
output_window: list = [30, 60, 90]

In [36]:
saved_models: list = []
for approach in multi_approaches:
    for input in input_window:
        for output in output_window:
            saved_models.append([f"saved_model/{approach}_i{input}_o{output}", input, output])



In [37]:
import tensorflow as tf
import os
import pickle
%run constants.py
def load_test_dataset(timestep:int, output:int):
    print(timestep)
    if timestep == 1:
        print("help")
        f = os.path.basename(X_TEST_INPUT_SAVE_FILE_VEC_MULTI_TIMESTEP)
        X_TEST_DIR = os.path.join(DATA_DIR, 'X_test_vec_single_timestep_30.pkl') 
        print(X_TEST_DIR)
        f = os.path.basename(Y_TEST_INPUT_SAVE_FILE)
        Y_TEST_DIR = os.path.join(DATA_DIR, os.path.splitext(os.path.basename(f))[0] + "_" + str(output) + os.path.splitext(os.path.basename(f))[1]) 
        print(Y_TEST_DIR)
    else:
        f = os.path.basename(X_TEST_INPUT_SAVE_FILE_VEC_MULTI_TIMESTEP)
        X_TEST_DIR = os.path.join(DATA_DIR, os.path.splitext(os.path.basename(f))[0] + "_" + str(timestep) + os.path.splitext(os.path.basename(f))[1]) 
        f = os.path.basename(Y_TEST_INPUT_SAVE_FILE)
        Y_TEST_DIR = os.path.join(DATA_DIR, os.path.splitext(os.path.basename(f))[0] + "_" + str(output) + os.path.splitext(os.path.basename(f))[1]) 
    print(X_TEST_DIR)
    with open(X_TEST_DIR, "rb") as f:
                x_test = pickle.load(f)

    with open(Y_TEST_DIR, "rb") as f:
                y_test = pickle.load(f)
    return x_test, y_test

results: list = []

import keras.backend as K
from sklearn.metrics import f1_score
import keras
import numpy
import tensorflow
from sklearn.metrics import plot_confusion_matrix

class estimator:
  _estimator_type = ''
  classes_=[]
  def __init__(self, model, classes):
    self.model = model
    self._estimator_type = 'classifier'
    self.classes_ = classes
  def predict(self, X):
    y_prob= self.model.predict(X)
    y_pred = y_prob.argmax(axis=1)
    return y_pred

from matplotlib import pyplot as plt
from sklearn.metrics import precision_recall_fscore_support, confusion_matrix

for model_info in saved_models:
    print(model_info[0])
    model = tf.keras.models.load_model(model_info[0])
    x_test, y_test = load_test_dataset(model_info[1], model_info[2])
    y_prob = model.predict(x_test)
    y_prob[y_prob <= 0.5] = 0
    y_prob[y_prob > 0.5] = 1
    y_prob = tensorflow.convert_to_tensor(y_prob)
    class_names = ['revist','non-revisit']
    classifier = estimator(model, class_names)
    y_prob = tf.cast(y_prob, tf.int64)
    y_test = tf.cast(y_test, tf.int64)
    y_prob = tf.reshape(y_prob,[-1])
    print(y_test.shape)
    print(y_prob.shape)
    cm = confusion_matrix(
    y_test,
    y_prob)
    results.append([model_info, model.evaluate(x_test, y_test), f1_score(y_test, y_prob), cm, precision_recall_fscore_support(y_test, y_prob, average='micro')])

saved_model/stacked_bilstm_i90_o30
90
/home/aaron/timeseries_nlp/data/X_test_vec_multi_timestep_90.pkl
22/22 [==============================] - 4s 88ms/step
(685,)
(685,)
22/22 [==============================] - 4s 93ms/step - loss: 0.6188 - binary_accuracy: 0.6599
saved_model/stacked_bilstm_i90_o60
90
/home/aaron/timeseries_nlp/data/X_test_vec_multi_timestep_90.pkl
22/22 [==============================] - 5s 88ms/step
(685,)
(685,)
22/22 [==============================] - 4s 93ms/step - loss: 0.6323 - binary_accuracy: 0.6307
saved_model/stacked_bilstm_i90_o90
90
/home/aaron/timeseries_nlp/data/X_test_vec_multi_timestep_90.pkl
22/22 [==============================] - 4s 88ms/step
(685,)
(685,)
22/22 [==============================] - 4s 93ms/step - loss: 0.6465 - binary_accuracy: 0.6073


In [38]:
for result in results:
    print(result)

results

[['saved_model/stacked_bilstm_i90_o30', 90, 30], [0.6188145875930786, 0.659853994846344], 0.5786618444846292, array([[292, 103],
       [130, 160]]), (0.6598540145985401, 0.6598540145985401, 0.6598540145985401, None)]
[['saved_model/stacked_bilstm_i90_o60', 90, 60], [0.6322587132453918, 0.6306569576263428], 0.6195488721804511, array([[226, 119],
       [134, 206]]), (0.6306569343065693, 0.6306569343065693, 0.6306569343065693, None)]
[['saved_model/stacked_bilstm_i90_o90', 90, 90], [0.6465084552764893, 0.607299268245697], 0.6492829204693612, array([[167, 134],
       [135, 249]]), (0.6072992700729927, 0.6072992700729927, 0.6072992700729927, None)]


[[['saved_model/stacked_bilstm_i90_o30', 90, 30],
  [0.6188145875930786, 0.659853994846344],
  0.5786618444846292,
  array([[292, 103],
         [130, 160]]),
  (0.6598540145985401, 0.6598540145985401, 0.6598540145985401, None)],
 [['saved_model/stacked_bilstm_i90_o60', 90, 60],
  [0.6322587132453918, 0.6306569576263428],
  0.6195488721804511,
  array([[226, 119],
         [134, 206]]),
  (0.6306569343065693, 0.6306569343065693, 0.6306569343065693, None)],
 [['saved_model/stacked_bilstm_i90_o90', 90, 90],
  [0.6465084552764893, 0.607299268245697],
  0.6492829204693612,
  array([[167, 134],
         [135, 249]]),
  (0.6072992700729927, 0.6072992700729927, 0.6072992700729927, None)]]